In [1]:
import nltk


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monju\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monju\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\monju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
import re

import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve,plot_precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA, TruncatedSVD
import time
import matplotlib.patches as mpatches
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import gc

from gensim.corpora import Dictionary
from gensim.models import LsiModel
gc.enable()

C:\tools\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
%%time 
posts=pd.read_csv('D:\so-data\PostsChosenV1.csv' , index_col='Id', )
# postTagsRawData = pd.read_csv('PostTagsV3.csv', chunksize=1000)
# postTagsRawData = pd.read_excel('PostsTags2.xlsx')
# print(posts)

Wall time: 4.55 s


<decorator-gen-55>:2: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.


In [6]:
#posts = posts.sample(10, replace=False).reset_index(drop=True)

In [4]:
from collections import defaultdict

def transform_text(text): 

    soup = BeautifulSoup(text)
    for elm in soup.find_all():
        if '<code>'== str(elm)[0:6]:
            elm.decompose()
    textStr = str(soup.get_text())
#     textStr = textStr.replace('<html>','')
#     textStr = textStr.replace('</html>','')
#     textStr = textStr.replace('<body>','')
#     textStr = textStr.replace('</body>','')
#     textStr = textStr.replace('<p>','')
#     textStr = textStr.replace('</p>','')
#     textStr = textStr.replace('<pre>','')
#     textStr = textStr.replace('</pre>','')
    
    
#     remove lines
    textStr = textStr.strip()
    # remove numbers
    textStr = re.sub(r"\d","", textStr)
    #lower document
    tokens = word_tokenize(textStr.lower())
    
    # remove stop words
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.',',','"',"'",'?','!',':', ";",', ',',  ',',   ', '(',')','[',']','{','}','#','...','--',"'s",'also',
                      '&','-','--','=','known','mi','km','$',"'",'`'])
    stopped_tokens = [token for token in tokens if token not in stop_words]

    
    # PorterStemmer is used to make the root word, like Paying to 'play'.
    ps = PorterStemmer()
    
    stemmer = []
    for s in stopped_tokens:
        if s.isalpha():
            stemmer.append(ps.stem(s))
#     stemmer = [s for s in stopped_tokens ps.stem(s)]
    
    lemmatizer = WordNetLemmatizer()
    
    lemmatized_tokens = [lemmatizer.lemmatize(i, pos="n") for i in stemmer]
    
    frequency = defaultdict(int)
    
    for lemmatized_token in lemmatized_tokens:
        frequency[lemmatized_token]+=1
    
    converted_lemmatized_tokens = []
            
    for lemmatized_token in lemmatized_tokens:
        if frequency[lemmatized_token]>1:
            converted_lemmatized_tokens.append(lemmatized_token)
    
    return converted_lemmatized_tokens

In [5]:
transformedPosts = []

for row in posts.itertuples(index=True, name='Pandas'):
    transformedPosts.append(transform_text(row.Body))

In [14]:
#create bigram and trigram models

from gensim.models import Phrases
bigram = Phrases(transformedPosts, min_count=10)
trigram = Phrases(bigram[transformedPosts])

In [15]:
for idx in range(len(transformedPosts)):
    for token in bigram[transformedPosts[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            transformedPosts[idx].append(token)
    for token in trigram[transformedPosts[idx]]:
        if '_' in token:
            # token is a bigram, add to document
            transformedPosts[idx].append(token)

In [11]:
print(transformedPosts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
from gensim import corpora

# remove rare & common tokens
dictionary = corpora.Dictionary(transformedPosts)
dictionary.filter_extremes(no_below=10, no_above=0.2)
corpus = [dictionary.doc2bow(text) for text in transformedPosts]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5802
Number of documents: 265020


In [13]:
print(dictionary)

Dictionary(26418 unique tokens: ['iphon', 'develop', 'mono', 'monotouch', 'applic']...)


In [ ]:
from gensim import models

num_of_topics= 30
passes = 20
iterations = 500
eval_every =1

model = models.LdaModel(corpus,
                        id2word=dictionary,
                        num_topics=num_of_topics,
                        iterations=iterations ,
                        alpha= 'auto',
                        eta = 'auto',
                        passes=passes,
                        eval_every= eval_every)

# Print the keyword in the topics
print(model.print_topics())

In [ ]:
# Computer Coherence Score using c_v

from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=model, texts=transformedPosts, dictionary=dictionary, coherence='c_v');
coherence_lda = coherence_model_lda.get_coherence()
print('Coherenence score: ', coherence_lda)

In [15]:
print(model.num_topics)

30


In [13]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_of_topics

print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -8.4508.
[([(0.2181874, 'build'),
   (0.112854734, 'io'),
   (0.07096896, 'understand'),
   (0.0657555, 'still'),
   (0.048403576, 'script'),
   (0.048058942, 'thing'),
   (0.045179512, 'sdk'),
   (0.030847738, 'cordova'),
   (0.030585982, 'use'),
   (0.026966073, 'simul'),
   (0.026060281, 'support'),
   (0.023148585, 'far'),
   (0.022667555, 'xcode'),
   (0.01959748, 'mean'),
   (0.018078996, 'unit'),
   (0.017885882, 'java'),
   (0.017271383, 'anyon'),
   (0.015516139, 'tri'),
   (0.013907216, 'built'),
   (0.009625933, 'appl')],
  -5.4616456938569975),
 ([(0.19115907, 'problem'),
   (0.13910592, 'add'),
   (0.11276023, 'see'),
   (0.08926561, 'log'),
   (0.074482, 'help'),
   (0.055466082, 'solut'),
   (0.047493923, 'tri'),
   (0.029861469, 'put'),
   (0.025062414, 'fix'),
   (0.022199398, 'point'),
   (0.019506998, 'theme'),
   (0.01921639, 'stack'),
   (0.018198619, 'handler'),
   (0.017144883, 'mode'),
   (0.01663413, 'noth'),
   (0.014954941, 'realli'),

In [28]:
for topic in top_topics:
    print('-----')
    print(topic[1])

-----
-4.152771539685277
-----
-4.467531967097778
-----
-4.703481903024933
-----
-4.973091260972155
-----
-5.010985856633441
-----
-5.020796581203571
-----
-5.052118048961289
-----
-5.054779218417982
-----
-5.094043234843933
-----
-5.169533845324151
-----
-5.180087517746917
-----
-5.182621207251478
-----
-5.192702982294745
-----
-5.201759430367363
-----
-5.227891350898298
-----
-5.239112708813488
-----
-5.433287838386504
-----
-5.435037810979641
-----
-5.573748761298403
-----
-5.645003825241395
-----
-5.662837828028232
-----
-5.69659806429641
-----
-5.8283028371632515
-----
-5.870085450164294
-----
-6.310128547874352
-----
-6.423212193901244
-----
-6.501154602638395
-----
-6.599260214735476
-----
-7.361401911041798
-----
-7.4828507061937835


In [18]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_apply',
 '_load_specials',
 '_save_specials',
 '_smart_save',
 'add_lifecycle_event',
 'alpha',
 'bound',
 'callbacks',
 'chunksize',
 'clear',
 'decay',
 'diff',
 'dispatcher',
 'distributed',
 'do_estep',
 'do_mstep',
 'dtype',
 'eta',
 'eval_every',
 'expElogbeta',
 'gamma_threshold',
 'get_document_topics',
 'get_term_topics',
 'get_topic_terms',
 'get_topics',
 'id2word',
 'inference',
 'init_dir_prior',
 'iterations',
 'lifecycle_events',
 'load',
 'log_perplexity',
 'minimum_phi_value',
 'minimum_probability',
 'num_terms',
 'num_topics',
 'num_updates',
 'numworkers',
 'offse